## Mount Drive

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
working_directory = '/content/drive/MyDrive/FireWatch'  # Update this path
os.chdir(working_directory)
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['grids.csv',
 'fires_with_grids.csv',
 'grids_with_pop_den_ids.csv',
 'weather.csv',
 'weather_locations.csv',
 'grids_within_weather_buffers.csv',
 'grids_with_weather_loc_id.csv',
 'fires_sample_2022.csv',
 'weather_with_loc.csv',
 'daily_weather.csv',
 'raw_data',
 'clean_data',
 'Firewatch-functions-to save.ipynb',
 'fires_sample_2023_8.csv',
 'sample_with_weather_2023-06-01_2023-08-30.csv',
 'weather_formatted.csv',
 'CO',
 'nlcd',
 '.ipynb_checkpoints',
 'Untitled0.ipynb']

# Create a 1 acre grid

In [3]:
import geopandas as gpd
import shapely.geometry as geom
import numpy as np
import math

def generate_sd_grid():
    # Download the US states shapefile from the U.S. Census Bureau.
    # (This zip file contains a shapefile of all states.)
    url = "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip"
    states = gpd.read_file(url)

    # Filter for South Dakota
    sd = states[states['NAME'] == 'South Dakota']

    # Reproject to an equal-area projection (EPSG:5070) in meters so that our area calculations are correct.
    sd = sd.to_crs(epsg=5070)

    # Define the side length of a 1-acre square in meters.
    # 1 acre ≈ 4046.8564 square meters, so side length = sqrt(4046.8564)
    cell_size = math.sqrt(4046.8564)

    # Get the bounding box of South Dakota in the projected coordinate system.
    minx, miny, maxx, maxy = sd.total_bounds

    # Create lists of coordinates for the grid cells.
    xs = np.arange(minx, maxx, cell_size)
    ys = np.arange(miny, maxy, cell_size)

    # Generate grid cells (as shapely polygons) covering the bounding box.
    grid_cells = [geom.box(x, y, x + cell_size, y + cell_size) for x in xs for y in ys]

    # Create a GeoDataFrame from the grid cells.
    grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs=sd.crs)

    # Clip the grid to only those cells that intersect South Dakota.
    grid_clipped = gpd.overlay(grid, sd, how='intersection')

    return grid_clipped

# Example usage:
if __name__ == "__main__":
    sd_grid = generate_sd_grid()
    print(sd_grid.head())
    # Optionally, save the grid to a file (e.g., as a shapefile or GeoPackage)
    # sd_grid.to_file("south_dakota_1acre_grid.shp")


KeyboardInterrupt: 

## Subset the grid for efficiency

In [ ]:
def select_western_quarter(grid):
    # Compute the total bounding box of the grid (in the grid's CRS)
    minx, miny, maxx, maxy = grid.total_bounds

    # Define the threshold x coordinate as 1/4 of the distance from minx to maxx
    x_threshold = minx + (maxx - minx) / 4

    # Select cells where the centroid's x coordinate is less than or equal to the threshold
    western_cells = grid[grid.geometry.centroid.x <= x_threshold]

    return western_cells

# Example usage:
if __name__ == "__main__":
    sd_grid = generate_sd_grid()  # assuming you have the grid generation function from earlier
    western_quarter = select_western_quarter(sd_grid)
    print(western_quarter.head())


# Query GEE by State for MODIS + Landsat + Treemap

*   MODIS NDVI, EVI, LST, DEM, Aspect & Slope
*   Harmonized Landsat & Sentinel-2
*   Treemap Biomass Characteristics






In [ ]:
import ee

# Initialize Earth Engine (authenticate if needed)
ee.Initialize()

def get_state_geometry(state_name):
    """
    Given a state name, returns its geometry from the TIGER/2018/States collection.
    """
    states = ee.FeatureCollection("TIGER/2018/States")
    state_feature = states.filter(ee.Filter.eq('NAME', state_name)).first()
    return state_feature.geometry()

def get_western_half_geometry(state_geometry):
    """
    Computes a rectangular region representing the western half of the state's bounding box.
    """
    state_bounds = state_geometry.bounds()
    coords = ee.List(state_bounds.coordinates().get(0))
    # Assume the first coordinate is bottom-left and the third is top-right.
    bl = ee.List(coords.get(0))  # [minx, miny]
    tr = ee.List(coords.get(2))  # [maxx, maxy]
    minx = ee.Number(bl.get(0))
    miny = ee.Number(bl.get(1))
    maxx = ee.Number(tr.get(0))
    maxy = ee.Number(tr.get(1))
    mid_x = minx.add(maxx).divide(2)
    western_half = ee.Geometry.Rectangle([minx, miny, mid_x, maxy])
    return western_half

def export_mod13a1_ndvi_evi(state_name, region, start_date, end_date):
    """
    Exports a mean composite of MODIS MOD13A1 NDVI and EVI for the specified region and time period.
    """
    mod13a1 = ee.ImageCollection("MODIS/006/MOD13A1") \
                .filterDate(start_date, end_date) \
                .select(['NDVI', 'EVI'])
    mod13a1_mean = mod13a1.mean().clip(region)

    state_clean = state_name.replace(" ", "_")
    file_prefix = f"mod13a1_{state_clean}"

    task = ee.batch.Export.image.toDrive(
        image=mod13a1_mean,
        description=f"{state_clean}_MOD13A1_mean_NDVI_EVI",
        folder='EarthEngineExports',
        fileNamePrefix=file_prefix,
        region=region.getInfo()['coordinates'],
        scale=500,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    print(f"Started export task for MOD13A1 NDVI/EVI for {state_name}.")
    return task

def export_mod11a1_lst(state_name, region, start_date, end_date):
    """
    Exports a mean composite of MODIS MOD11A1 LST for the specified region and time period.
    """
    mod11a1 = ee.ImageCollection("MODIS/006/MOD11A1") \
                .filterDate(start_date, end_date) \
                .select('LST_Day_1km')
    mod11a1_mean = mod11a1.mean().clip(region)

    state_clean = state_name.replace(" ", "_")
    file_prefix = f"mod11a1_{state_clean}"

    task = ee.batch.Export.image.toDrive(
        image=mod11a1_mean,
        description=f"{state_clean}_MOD11A1_mean_LST",
        folder='EarthEngineExports',
        fileNamePrefix=file_prefix,
        region=region.getInfo()['coordinates'],
        scale=1000,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    print(f"Started export task for MOD11A1 LST for {state_name}.")
    return task

def export_dem_slope_aspect(state_name, region):
    """
    Exports DEM, slope, and aspect derived from the SRTM DEM for the specified region.
    """
    dem = ee.Image("USGS/SRTMGL1_003").clip(region)
    slope = ee.Terrain.slope(dem)
    aspect = ee.Terrain.aspect(dem)
    dem_aspect_slope = dem.select([0], ['DEM']) \
                          .addBands(slope.rename('slope')) \
                          .addBands(aspect.rename('aspect'))

    state_clean = state_name.replace(" ", "_")
    file_prefix = f"dem_slope_aspect_{state_clean}"

    task = ee.batch.Export.image.toDrive(
        image=dem_aspect_slope,
        description=f"{state_clean}_DEM_Slope_Aspect",
        folder='EarthEngineExports',
        fileNamePrefix=file_prefix,
        region=region.getInfo()['coordinates'],
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    print(f"Started export task for DEM, slope, and aspect for {state_name}.")
    return task

def export_hls(state_name, region, start_date, end_date):
    """
    Exports a median composite of the harmonized Landsat-Sentinel-2 (HLS) product for the specified region and time period.
    """
    hls = ee.ImageCollection("projects/USDA/NASS/HLS") \
            .filterDate(start_date, end_date) \
            .filterBounds(region) \
            .median() \
            .clip(region)

    state_clean = state_name.replace(" ", "_")
    file_prefix = f"hls_{state_clean}"

    task = ee.batch.Export.image.toDrive(
        image=hls,
        description=f"{state_clean}_HLS_median",
        folder='EarthEngineExports',
        fileNamePrefix=file_prefix,
        region=region.getInfo()['coordinates'],
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    print(f"Started export task for HLS for {state_name}.")
    return task

def export_treemap(state_name, region):
    """
    Exports the Treemap layers for the specified region using the
    ee.ImageCollection('USFS/GTAC/TreeMap/v2016') asset. The layers include:
      - BALIVE (Live Tree Basal Area)
      - CANOPYPCT (Live Canopy Cover)
      - CARBON_D (Carbon, Standing Dead)
      - CARBON_DWN (Carbon, Down Dead)
      - DRYBIO_D (Dry Standing Dead Tree Biomass)
      - DRYBIO_L (Dry Live Tree Biomass)
      - FLDSZCD (Field Stand-Size Class Code)
    """
    treemap_ic = ee.ImageCollection('USFS/GTAC/TreeMap/v2016').filterBounds(region)
    treemap = treemap_ic.mosaic() \
                .select(["BALIVE", "CANOPYPCT", "CARBON_D", "CARBON_DWN", "DRYBIO_D", "DRYBIO_L", "FLDSZCD"]) \
                .clip(region)

    state_clean = state_name.replace(" ", "_")
    file_prefix = f"treemap_{state_clean}"

    task = ee.batch.Export.image.toDrive(
        image=treemap,
        description=f"{state_clean}_TREEMAP_layers",
        folder='EarthEngineExports',
        fileNamePrefix=file_prefix,
        region=region.getInfo()['coordinates'],
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    print(f"Started export task for Treemap layers for {state_name}.")
    return task

if __name__ == "__main__":
    # User-specified parameters
    state_name = "South Dakota"  # Change this to any valid state name from TIGER/2018/States
    start_date = '2020-01-01'
    end_date = '2020-12-31'

    # Retrieve the state geometry and compute the western half of the state's bounding box.
    state_geom = get_state_geometry(state_name)
    region = get_western_half_geometry(state_geom)

    # Launch export tasks for each dataset.
    task1 = export_mod13a1_ndvi_evi(state_name, region, start_date, end_date)
    task2 = export_mod11a1_lst(state_name, region, start_date, end_date)
    task3 = export_dem_slope_aspect(state_name, region)
    task4 = export_hls(state_name, region, start_date, end_date)
    task5 = export_treemap(state_name, region)

    print("All export tasks have been started. Monitor the Earth Engine Tasks tab for progress.")


# Generate Indices from Landsat/Sentinel Data

In [ ]:


def calculate_hls_indices(hls_image):
    """
    Calculates a suite of vegetation and fire-related indices from a harmonized Landsat-Sentinel-2 (HLS) image.
    Assumes the following bands are present in the image:
      - 'B' for Blue
      - 'G' for Green
      - 'R' for Red
      - 'N' for Near-Infrared (NIR)
      - 'RE' for Red Edge
      - 'SWIR1' for Shortwave Infrared

    The calculated indices are:
      1. NDVI: (N - R) / (N + R)
      2. GNDVI: (N - G) / (N + G)
      3. BNDVI: (N - B) / (N + B)
      4. LCI: (N / G) - 1
      5. MCARI: [ (RE - R) - 0.2*(R - G) ] * (RE / R)
      6. NDRE: (N - RE) / (N + RE)
      7. NDWI: (N - SWIR1) / (N + SWIR1)
      8. SIPI: (N - B) / (N - R)
      9. TGI: -0.5 * [660*(R - G) - 450*(R - B)]
      10. VARI: (G - R) / (G + R - B)
      11. NBR: (N - SWIR1) / (N + SWIR1)

    Returns:
      An ee.Image with each index as a separate band.
    """
    # 1. NDVI = (N - R) / (N + R)
    ndvi = hls_image.normalizedDifference(['N', 'R']).rename('NDVI')

    # 2. GNDVI = (N - G) / (N + G)
    gndvi = hls_image.normalizedDifference(['N', 'G']).rename('GNDVI')

    # 3. BNDVI = (N - B) / (N + B)
    bndvi = hls_image.normalizedDifference(['N', 'B']).rename('BNDVI')

    # 4. LCI = (N / G) - 1
    lci = hls_image.select('N').divide(hls_image.select('G')).subtract(1).rename('LCI')

    # 5. MCARI = [ (RE - R) - 0.2*(R - G) ] * (RE / R)
    mcari = hls_image.expression(
        '((RE - R) - 0.2 * (R - G)) * (RE / R)',
        {
            'RE': hls_image.select('RE'),
            'R': hls_image.select('R'),
            'G': hls_image.select('G')
        }
    ).rename('MCARI')

    # 6. NDRE = (N - RE) / (N + RE)
    ndre = hls_image.normalizedDifference(['N', 'RE']).rename('NDRE')

    # 7. NDWI = (N - SWIR1) / (N + SWIR1)
    ndwi = hls_image.normalizedDifference(['N', 'SWIR1']).rename('NDWI')

    # 8. SIPI = (N - B) / (N - R)
    sipi = hls_image.expression(
        '(N - B) / (N - R)',
        {
            'N': hls_image.select('N'),
            'B': hls_image.select('B'),
            'R': hls_image.select('R')
        }
    ).rename('SIPI')

    # 9. TGI = -0.5 * [660*(R - G) - 450*(R - B)]
    tgi = hls_image.expression(
        '-0.5 * (660 * (R - G) - 450 * (R - B))',
        {
            'R': hls_image.select('R'),
            'G': hls_image.select('G'),
            'B': hls_image.select('B')
        }
    ).rename('TGI')

    # 10. VARI = (G - R) / (G + R - B)
    vari = hls_image.expression(
        '(G - R) / (G + R - B)',
        {
            'G': hls_image.select('G'),
            'R': hls_image.select('R'),
            'B': hls_image.select('B')
        }
    ).rename('VARI')

    # 11. NBR = (N - SWIR1) / (N + SWIR1)
    nbr = hls_image.normalizedDifference(['N', 'SWIR1']).rename('NBR')

    # Combine all indices into one multi-band image.
    indices = ndvi.addBands(gndvi) \
                  .addBands(bndvi) \
                  .addBands(lci) \
                  .addBands(mcari) \
                  .addBands(ndre) \
                  .addBands(ndwi) \
                  .addBands(sipi) \
                  .addBands(tgi) \
                  .addBands(vari) \
                  .addBands(nbr)

    return indices

def export_hls_indices(hls_image, region, file_prefix, scale=30):
    """
    Calculates indices from the provided HLS image and exports the results as a GeoTIFF to Google Drive.

    Parameters:
      hls_image: The harmonized Landsat-Sentinel-2 image (from Task 4 of your earlier query scripts).
      region: An ee.Geometry defining the export region.
      file_prefix: A string to be used as part of the file name (e.g., a state name).
      scale: The export resolution in meters (default is 30).

    Returns:
      The export task.
    """
    # Compute the indices image.
    indices_image = calculate_hls_indices(hls_image)

    # Set up and start the export task.
    task = ee.batch.Export.image.toDrive(
        image=indices_image,
        description=f"{file_prefix}_HLS_Indices",
        folder='EarthEngineExports',
        fileNamePrefix=f"{file_prefix}_HLS_Indices",
        region=region.getInfo()['coordinates'],
        scale=scale,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    print(f"Export task for HLS indices started with prefix: {file_prefix}_HLS_Indices")
    return task

# Example usage:
if __name__ == "__main__":
    # In your workflow, the hls_image should come from Task 4 in your earlier query scripts.
    # For this example, we create a dummy HLS composite by filtering the HLS collection.
    start_date = '2020-01-01'
    end_date = '2020-12-31'

    # Define an example region; in your case, use your region of interest.
    region = ee.Geometry.Rectangle([-102.0, 43.0, -101.0, 44.0])

    # Retrieve the HLS image (from Task 4) by creating a median composite.
    hls_collection = ee.ImageCollection("projects/USDA/NASS/HLS") \
                        .filterDate(start_date, end_date) \
                        .filterBounds(region)
    hls_image = hls_collection.median().clip(region)

    # Define the file prefix (for example, the state name).
    file_prefix = "ExampleState"  # Replace with your desired prefix.

    # Calculate indices and export as a TIFF.
    export_task = export_hls_indices(hls_image, region, file_prefix, scale=30)


# GOES GLM Lightning

# Snotel data

# MERRA Airquality data

# Wildfire foot prints by state

In [ ]:
import ee

# Initialize Earth Engine (authenticate if needed)
ee.Initialize()

def get_state_geometry(state_name):
    """
    Retrieves the geometry for a given state from the TIGER/2018/States collection.
    """
    states = ee.FeatureCollection("TIGER/2018/States")
    state_feature = states.filter(ee.Filter.eq('NAME', state_name)).first()
    return state_feature.geometry()

def get_wildfire_footprints_by_state(state_name, start_date, end_date, scale=500):
    """
    Retrieves wildfire footprints for a specified state and time period using
    the MODIS Burned Area product (MCD64A1). The function creates a composite
    of the burned area images, generates a binary burned mask (pixels where
    BurnDate > 0), and converts the mask into polygon features.

    Parameters:
      state_name: A string (e.g., "California").
      start_date: The start date (YYYY-MM-DD) for the query.
      end_date: The end date (YYYY-MM-DD) for the query.
      scale: The spatial resolution in meters (default is 500 m).

    Returns:
      An ee.FeatureCollection of wildfire footprint polygons.
    """
    # Get the state geometry.
    state_geom = get_state_geometry(state_name)

    # Load the MODIS burned area collection and filter by date and state.
    burned_collection = ee.ImageCollection("MODIS/006/MCD64A1") \
                          .filterDate(start_date, end_date) \
                          .filterBounds(state_geom)

    # Mosaic the collection into one image and clip to the state.
    burned_image = burned_collection.mosaic().clip(state_geom)

    # Create a binary mask where BurnDate > 0 (i.e., burned pixels).
    burned_mask = burned_image.select("BurnDate").gt(0)

    # Convert the binary burned mask to vectors (polygons).
    wildfire_footprints = burned_mask.reduceToVectors(
        geometry=state_geom,
        scale=scale,
        geometryType='polygon',
        eightConnected=True,
        labelProperty='burned',
        reducer=ee.Reducer.first()
    )

    return wildfire_footprints

def export_wildfire_footprints(state_name, start_date, end_date, scale=500):
    """
    Exports wildfire footprints for a given state and time period as a shapefile to Google Drive.

    Parameters:
      state_name: A string (e.g., "California").
      start_date: The start date (YYYY-MM-DD) for the query.
      end_date: The end date (YYYY-MM-DD) for the query.
      scale: The spatial resolution in meters (default is 500 m).

    Returns:
      The export task.
    """
    footprints = get_wildfire_footprints_by_state(state_name, start_date, end_date, scale)
    state_clean = state_name.replace(" ", "_")

    task = ee.batch.Export.table.toDrive(
        collection=footprints,
        description=f"{state_clean}_Wildfire_Footprints",
        folder='EarthEngineExports',
        fileNamePrefix=f"{state_clean}_Wildfire_Footprints",
        fileFormat='SHP'
    )
    task.start()
    print(f"Export task for wildfire footprints of {state_name} started.")
    return task

if __name__ == "__main__":
    # Example parameters (adjust as needed)
    state_name = "California"  # Change to your state of interest
    start_date = "2020-01-01"
    end_date = "2020-12-31"

    # Retrieve wildfire footprints and print the number of features
    footprints = get_wildfire_footprints_by_state(state_name, start_date, end_date)
    print("Number of wildfire footprint features:", footprints.size().getInfo())

    # Export the wildfire footprints to Google Drive as a shapefile.
    export_task = export_wildfire_footprints(state_name, start_date, end_date)
